In [191]:
import csv
import numpy as np
import re

maxsize = 10000 # 最多处理的数据行数

filename = "数据集_找到网人力资源数据.csv"
with open(filename,'r',encoding='GB18030') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    clen = len(headers) # 列长度
    cnt = 0
    data_ori = []
    for row in f_csv:
        cnt += 1
        # 边缘空格的处理
        for j in range(clen):
            row[j] = row[j].strip()
        data_ori.append(row)
        if(cnt == maxsize):
            break;
    data = np.array(data_ori)

rlen = len(data)    # 行长度

dict = {}       # 创建字典来保存列数据,可通过中文名称访问，便于后续处理
for i in range(len(headers)):
    dict[headers[i]] = data[:,i]



In [101]:
# 处理电话号码
def phone_check(str):
    pattern = re.compile(r'^(13[0-9]|14[0|5|6|7|9]|15[0|1|2|3|5|6|7|8|9]|'
                            r'16[2|5|6|7]|17[0|1|2|3|5|6|7|8]|18[0-9]|'
                            r'19[1|3|5|6|7|8|9])\d{8}$')
    return bool(pattern.search(str))

for i in range(rlen):
    if phone_check( dict['tel'][i] )==1 or dict['tel'][i] == '':
        continue
    else:
        print(i,dict['tel'][i])
        # dict['tel'][i] = ''

5602 973387486


In [102]:
# 处理邮件
def email_check(str):
    pattern = re.compile(r'^\w+([-+.]\w+)*@\w+([-.]\w+)*\.\w+([-.]\w+)*$')

    return bool(pattern.search(str))

for i in range(rlen):
    if email_check( dict['email'][i] )==1 or dict['email'][i] == '':
        continue
    else:
        print(i,dict['email'][i])
        dict['email'][i] = ''

In [163]:
# 处理注册资金
def money_check(str):
    pattern = re.compile(r'[ \u5143 | \u4e07 | \u4ebf]$')   # 以元，万，亿结尾

    return bool(pattern.search(str))

money = []  # 记录数字形式的资金

for i in range(rlen):
    if money_check( dict['注册资金'][i] )==1:
        dict['注册资金'][i] = dict['注册资金'][i].replace('元','')
        dict['注册资金'][i] = dict['注册资金'][i].replace('万','e4')
        dict['注册资金'][i] = dict['注册资金'][i].replace('亿','e8')
        money.append(eval(dict['注册资金'][i]))
    else:   # 不合规范或者为空的时候
        money.append(0)
    
dict['注册资金'] = np.array(money).astype(int)  # 由str变为int数组



# 数据概化
# 注册资金等级
money_level = ["1000万以下","1000万以上5000万以下","5000万以上1亿以下","1亿以上"]
money_digit = []
for i in range(rlen):
    if dict['注册资金'][i] <= 1e7:
        money_digit.append(0)
    elif dict['注册资金'][i] <= 5e7:
        money_digit.append(1)
    elif dict['注册资金'][i] <= 1e8:
        money_digit.append(2)
    else:
        money_digit.append(3)

dict['注册资金等级'] = np.array(money_digit)

In [192]:
# 部门-职务
depart = []
for i in range(rlen):
    depart.append(dict['部门-职务'][i].strip().split(" "))
    if (len(depart[i])<2):
        depart[i].append("")
    if (len(depart[i])>2):
        del depart[i][0:len(depart[i])-2]

dict['部门-职务'] = np.array(depart)

In [ ]:
# 职业等级 job_level
job_level = ['a','b','c','d','e']
pattern_jla = re.compile(r'董事|主席')
pattern_jlb = re.compile(r'总经理|总裁|副总经理')
pattern_jlc = re.compile(r'总监|副总监|经理|副经理|主任|主管')
pattern_jld = re.compile(r'工程师|\w{1,2}员$|实习生')
pattern_job_level = [pattern_jla,pattern_jlb,pattern_jlc,pattern_jld]
job_level_temp = []
for i in range(rlen):
    flag = 0
    for j in range(len(job_level)-1):
        if (pattern_job_level[j].search(dict['部门-职务'][i,1])):
            job_level_temp.append(job_level[j])
            flag = 1
            break
    if flag == 0:
        job_level_temp.append('e')

dict['职务等级'] = np.array(job_level_temp)
# print(len(job_level_temp))
# for i in range(rlen):
#     print(dict['部门-职务'][i,1],dict['职务等级'][i])

In [ ]:
# 工作类别 job_class
job_class = ["市场类","技术类","营销类","其他类"]
pattern_jca = re.compile(r'销售|市场|客户')
pattern_jcb = re.compile(r'业务|技术|项目')
pattern_jcc = re.compile(r'营销|宣传')
pattern_jcd = re.compile(r'财务|运营|行政|人力')
pattern_job_class = [pattern_jca,pattern_jcb,pattern_jcc,pattern_jcd]
job_class_temp = []
for i in range(rlen):
    flag = 0
    for j in range(len(job_class)-1):
        if (pattern_job_class[j].search(dict['部门-职务'][i,0])):
            job_class_temp.append(job_class[j])
            flag = 1
            break
    if flag == 0:
        job_class_temp.append("其他类")

dict['工作类别'] = np.array(job_class_temp)
# print(len(job_class_temp))
# for i in range(rlen):
#     print(dict['部门-职务'][i,0],dict['工作类别'][i])